#Импорт библиотек

In [174]:
import nltk
import string
import csv
import pandas as pd
from google.colab import files 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#Функция токенезациии
На вход поступает текст кода --> на выход возвращается массив токенов слова

In [175]:
def tokenization(filename):
    #file = csv.reader(open(filename, encoding="utf8"), delimiter=",")
    tokens = []
    #разбивка на токены:
    #for line in filename:
    tokens += nltk.word_tokenize(str(filename))
    #очистка от знаков препинания:
    tokens = [i for i in tokens if (i not in string.punctuation)]
    #очистка от мусора:
    tokens = [i.replace("'", "").replace("`", "").replace("=", "").replace("/", "").replace("\\", "").replace("-", "").replace(":", "").replace("|", "") for i in tokens]
    #очистка от пустых элементов и элементов длинной <= 2:
    tokens = [i for i in tokens if len(i) >3]
    return tokens

###Пример работы



In [183]:
a = '{//! [0]QSqlDatabase db = QSqlDatabase::addDatabase("QPSQL") Teddy1 ;db.setHostName("acidalia");db.setDatabaseName("customdb");db.setUserName("mojito");db.setPassword("");bool ok = db.open();//! [0]Q_UNUSED(ok);}{//! [1]'
context = tokenization(a)
context

['QSqlDatabase',
 'QSqlDatabase',
 'addDatabase',
 'QPSQL',
 'Teddy1',
 'db.setHostName',
 'acidalia',
 'db.setDatabaseName',
 'customdb',
 'db.setUserName',
 'mojito',
 'db.setPassword',
 'bool',
 'db.open',
 'Q_UNUSED']

#Функция составления окружения
На вход поступают:
* password - потенциальный пароль (полученный из морфологической ML модели)
* context - текст кода, в которох находится password
* ntokens_max - количество слов окружения в одну и другую сторону.

На выходе возвращается массив окружения пароля длиной 2n токенов.




In [181]:
def insertion_pass(password, context, ntokens_max = 10):
  result = []
  for i in context:

    if i == password:

      #вперед

      if (len(context) - context.index(i)) <= ntokens_max:
        ntokens_f = len(context) - context.index(i) - 1
      else:
        ntokens_f = ntokens_max

      count_forward = context.index(i) + 1

      for j in range(ntokens_f):
        result.append(context[count_forward])
        count_forward += 1

      #назад

      if context.index(i) < ntokens_max:
        ntokens_b = context.index(i)
      else:
        ntokens_b = ntokens_max
        
      count_backward = context.index(i) - 1

      for j in range(ntokens_b):
        result.insert(0, context[count_backward])
        count_backward -= 1

  return result

###Пример работы

In [186]:
a = '{//! [0]QSqlDatabase db = QSqlDatabase::addDatabase("QPSQL") Teddy1 ;db.setHostName("acidalia");db.setDatabaseName("customdb");db.setUserName("mojito");db.setPassword("");bool ok = db.open();//! [0]Q_UNUSED(ok);}{//! [1]'
password = 'Teddy1'
context = tokenization(a)
print(insertion_pass(password, context, 10))

['QSqlDatabase', 'QSqlDatabase', 'addDatabase', 'QPSQL', 'db.setHostName', 'acidalia', 'db.setDatabaseName', 'customdb', 'db.setUserName', 'mojito', 'db.setPassword', 'bool', 'db.open', 'Q_UNUSED']


#Функция определения позиции пароля
На вход поступает датафрейм после работы морфологической ML модели --> На выходе возвращается датафрейм содержащий контекст потенциального пароля.

In [179]:
def Position_Determination(dataframe):
  test_data = dataframe['Input'][dataframe['Target'] == 1].tolist()
  massive_pass = []
  for i in test_data:
    for j in insertion_pass(i, context, 10):
        massive_pass.append(j)
  stroka = [' '.join(massive_pass)]
  return pd.DataFrame(stroka)

###Пример работы

In [ ]:
a = '{//! [0]QSqlDatabase db = QSqlDatabase::addDatabase("QPSQL") Teddy1 ;db.setHostName("acidalia");db.setDatabaseName("customdb");db.setUserName("mojito");db.setPassword("");bool ok = db.open();//! [0]Q_UNUSED(ok);}{//! [1]'
context = tokenization(a)
df = pd.read_csv('https://www.dropbox.com/s/ny23cfp6b6pb6aa/password_dataset.csv?dl=1')
Position_Determination(df)

In [191]:
df = pd.read_csv('https://www.dropbox.com/s/ny23cfp6b6pb6aa/password_dataset.csv?dl=1')
df

,Input,Length,Uppercase,Lowercase,Digits,Special,Entropy,Target
0,Usuckballz1,11,0.09,0.82,0.09,0.00,2.14,1
1,Soso123aljg,11,0.09,0.64,0.27,0.00,2.44,1
2,Mailcreated5240,15,0.07,0.67,0.27,0.00,3.32,1
3,Passw0rd,8,0.12,0.75,0.12,0.00,1.60,1
4,8J4yE3Uz,8,0.38,0.25,0.38,0.00,1.90,1
...,...,...,...,...,...,...,...,...
981960,T7177306a.javan,15,0.07,0.40,0.47,0.07,3.75,0
981961,bytes_in,8,0.00,0.88,0.00,0.12,1.42,0
981962,packets_out,11,0.00,0.91,0.00,0.09,1.96,0
981963,nnuart1,7,0.00,0.86,0.14,0.00,1.42,0


In [190]:
a = '{//! [0]QSqlDatabase db = QSqlDatabase::addDatabase("QPSQL") Teddy1 ;db.setHostName("acidalia");db.setDatabaseName("customdb");db.setUserName("mojito");db.setPassword("");bool ok = db.open();//! [0]Q_UNUSED(ok);}{//! [1]'
context = tokenization(a)
context

['QSqlDatabase',
 'QSqlDatabase',
 'addDatabase',
 'QPSQL',
 'Teddy1',
 'db.setHostName',
 'acidalia',
 'db.setDatabaseName',
 'customdb',
 'db.setUserName',
 'mojito',
 'db.setPassword',
 'bool',
 'db.open',
 'Q_UNUSED']

In [189]:
Position_Determination(df)

,0
0,QSqlDatabase QSqlDatabase addDatabase QPSQL db...
